In [1]:
#Import necessary librarys for this section
#Open first data set - 5 core reviews for electronics - by converting JSON file to pandas dataframe format 

import pandas as pd
import gzip
from datetime import datetime

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Electronics_5.json.gz')
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [2]:
#Look at the length of the total dataset 
print(len(df))

1689188


In [3]:
#Analyse the format of all data types 
#Most are labelled as objects apart from the rating and Time which are numerical
#Time needs to be changed to an appropriate format 
df.dtypes

reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
dtype: object

In [4]:
#changing the Time format to pandas DateTime
df['ReviewTime'] = pd.to_datetime(df['unixReviewTime'],unit='s')

In [5]:
#A new column called ReviewTime was added to the dataframe with correct format
df.dtypes

reviewerID                object
asin                      object
reviewerName              object
helpful                   object
reviewText                object
overall                  float64
summary                   object
unixReviewTime             int64
reviewTime                object
ReviewTime        datetime64[ns]
dtype: object

In [6]:
#The other two columms with incorrect format for Date/Time were removed from the dataframe 
df.drop(['reviewTime', 'unixReviewTime'], axis="columns", inplace =True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,ReviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,2013-06-02
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,2010-11-25
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,2010-09-09
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",2010-11-24
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",2011-09-29


In [7]:
#We want to keep duplicate rows as there are multiple of the same products but with different ratings.
#To make analysis easier we need to change the product name (asin) to the actual product name
#To change product name we need the metadata for the ratings and merge tables together 

In [8]:
#Open up next file which contains the metadata for the electronics category


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

metadata = getDF('meta_Electronics.json.gz')
metadata.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [9]:
#Merge both dataframes together using the column 'asin' as a common key from both 
rating_data = pd.merge(df, metadata, how="left", on="asin")
rating_data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,ReviewTime,imUrl,description,categories,title,price,salesRank,related,brand
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,2013-06-02,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,2010-11-25,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,2010-09-09,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",2010-11-24,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",2011-09-29,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [10]:
#Visualise if any of the columns contain null values
rating_data.isnull().sum()

reviewerID           0
asin                 0
reviewerName     24730
helpful              0
reviewText           0
overall              0
summary              0
ReviewTime           0
imUrl             1213
description      33677
categories           0
title            45502
price            49306
salesRank       879118
related          27046
brand           734937
dtype: int64

In [11]:
#Drop all rows that contain null values as they create inaccuracy in data reliance 
rating_data = rating_data.dropna()

In [12]:
#Double checking the dataset contains no null values 
rating_data.isnull().sum()

reviewerID      0
asin            0
reviewerName    0
helpful         0
reviewText      0
overall         0
summary         0
ReviewTime      0
imUrl           0
description     0
categories      0
title           0
price           0
salesRank       0
related         0
brand           0
dtype: int64

In [13]:
#Visualising all formats for merge data set
rating_data.dtypes

reviewerID              object
asin                    object
reviewerName            object
helpful                 object
reviewText              object
overall                float64
summary                 object
ReviewTime      datetime64[ns]
imUrl                   object
description             object
categories              object
title                   object
price                  float64
salesRank               object
related                 object
brand                   object
dtype: object

In [14]:
#Some columns are not necessary for the analysis and hence removed to make the data easier to work with 
rating_data.drop(['imUrl', 'reviewerName', 'salesRank'], axis="columns", inplace =True)
rating_data.head()

,reviewerID,asin,helpful,reviewText,overall,summary,ReviewTime,description,categories,title,price,related,brand
19,A2IDCSC6NVONIZ,0972683275,"[1, 1]",This mount is just what I needed. It is stron...,5.0,Perfect,2013-04-30,The VideoSecu TV mount is a mounting solution ...,"[[Electronics, Accessories & Supplies, Audio &...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu
20,A1EDI0X3GI1SK7,0972683275,"[0, 1]","Great deal, easy to mount and it appears to be...",5.0,32 In TV Mount,2011-12-16,The VideoSecu TV mount is a mounting solution ...,"[[Electronics, Accessories & Supplies, Audio &...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu
21,A3BMUBUC1N77U8,0972683275,"[0, 0]",This mount works really well once you get it u...,4.0,"Pretty simple, but definitely good!",2013-11-23,The VideoSecu TV mount is a mounting solution ...,"[[Electronics, Accessories & Supplies, Audio &...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu
22,AVRFGGCCCR6QU,0972683275,"[3, 4]",This wall mount does everything it's supposed ...,4.0,Fairly good wall mount,2010-08-30,The VideoSecu TV mount is a mounting solution ...,"[[Electronics, Accessories & Supplies, Audio &...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu
23,A3UOSOCRKS3WIH,0972683275,"[0, 0]",for the price you just cant beat this item. I ...,5.0,Cant Beat it!,2013-05-12,The VideoSecu TV mount is a mounting solution ...,"[[Electronics, Accessories & Supplies, Audio &...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu


In [15]:
#Final length of the data 
print(len(rating_data))

517687


In [ ]:
#Saving dataframe to csv file for part two of the assignment 
rating_data.to_csv(r'C:\Users\emjha\3030ICT.data.csv', index=False)